In [ ]:
import Pkg; Pkg.instantiate()
Pkg.activate(".")
using DrugResponseModel
concs, _, g1s1, g2s1 = load(189, 1);
_, _, g1s2, g2s2 = load(189, 2);
_, _, g1s3, g2s3 = load(189, 3);
g0 = g1s1[1,1,1]+g2s1[1,1,1];
g1m = (g1s1 .+ g1s2 .+ g1s3) ./ 3;
g2m = (g2s1 .+ g2s2 .+ g2s3) ./ 3;


In [ ]:
# case 1

function getODEparamspairs(p, conc)
    effects = zeros(eltype(p), 9, length(conc[:, 1]), 2)
    k = 1
    # Scaled drug effect
    for i = 1:2
        xx = 1.0 ./ (1.0 .+ (p[k] ./ conc[:, i]) .^ p[k + 1])
        effects[1, :, i] = p[15] .+ (p[k + 2] - p[36]) .* xx # G1 prog. rate
        effects[2, :, i] = p[37] .+ (p[k + 3] - p[37]) .* xx # G2 prog. rate
        effects[3, :, i] = p[k + 4] .* xx # G1 death rate
        effects[4, :, i] = p[k + 5] .* xx # G2 death rate
        effects[5, :, i] .= p[k + 6] # percentage in G1
        effects[6, :, i] .= p[17]
        effects[7, :, i] .= p[18]
        effects[8, :, i] .= p[19]
        effects[9, :, i] .= p[20]
        
        k += 13
    end
    return effects
end

a, p1 = DrugResponseModel.BBoptim_DrugPairs(concs, g1m, g2m, 1, 1, 2)
pp1 = getODEparamspairs(p1, concs)



In [ ]:

# case 2
function getODEparamspairs(p, conc)
    effects = zeros(eltype(p), 9, length(conc[:, 1]), 2)
    k = 1
    # Scaled drug effect
    for i = 1:2
        xx = 1.0 ./ (1.0 .+ (p[k] ./ conc[:, i]) .^ p[k + 1])
        effects[1, :, i] = p[k + 2] .+ (p[k + 3] - p[k + 2]) .* xx # G1 prog. rate
        effects[2, :, i] = p[k + 4] .+ (p[k + 5] - p[k + 4]) .* xx # G2 prog. rate
        effects[3, :, i] = p[k + 6] .* xx # G1 death rate
        effects[4, :, i] = p[k + 7] .* xx # G2 death rate
        effects[5, :, i] .= p[k + 8] # percentage in G1
        effects[6, :, i] .= p[19]
        effects[7, :, i] .= p[20]
        effects[8, :, i] .= p[21]
        effects[9, :, i] .= p[22]
        
        k += 9
    end
    return effects
end

a, p2 = DrugResponseModel.BBoptim_DrugPairs(concs, g1m, g2m, 2, 1, 2)
pp2 = getODEparamspairs(p2, conc)





In [ ]:
# case 3
function getODEparamspairs(p, conc)
    effects = zeros(eltype(p), 9, length(conc[:, 1]), 2)
    k = 1
    # Scaled drug effect
    for i = 1:2
        xx = 1.0 ./ (1.0 .+ (p[k] ./ conc[:, i]) .^ p[k + 1])
        effects[1, :, i] = p[23] .+ (p[k + 2] - p[23]) .* xx # G1 prog. rate
        effects[2, :, i] = p[24] .+ (p[k + 3] - p[24]) .* xx # G2 prog. rate
        effects[3, :, i] = p[k + 4] .* xx # G1 death rate
        effects[4, :, i] = p[k + 5] .* xx # G2 death rate
        effects[5, :, i] .= p[k + 6] # percentage in G1
        effects[6, :, i] .= p[k + 7]
        effects[7, :, i] .= p[k + 8]
        effects[8, :, i] .= p[k + 9]
        effects[9, :, i] .= p[k + 10]
        
        k += 11
    end
    return effects
end

a, p3 = DrugResponseModel.BBoptim_DrugPairs(concs, g1m, g2m, 3, 1, 2)
pp3 = getODEparamspairs(p3, conc)


